In [63]:
import lpips
import glob
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load the pre-trained LPIPS model
loss_fn = lpips.LPIPS(net='alex')

# Path to the directories containing the images of the two datasets
dataset1_dir = './data1'
dataset2_dir = './data2'

# Get the paths of the images in the two datasets
dataset1_images = glob.glob(dataset1_dir + '/*.png')
dataset2_images = glob.glob(dataset2_dir + '/*.png')

# Calculate LPIPS for each pair of images
total_lpips = 0.0
num_pairs = min(len(dataset1_images), len(dataset2_images))

if num_pairs > 0:
    for i in range(num_pairs):
        # Load images
        image1 = cv2.imread(dataset1_images[i])
        image2 = cv2.imread(dataset2_images[i])

        # Convert images to PIL Image format
        image1_pil = Image.fromarray(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
        image2_pil = Image.fromarray(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))

        # Resize images to a common size
        transform = transforms.Resize((256, 256))
        image1_resized = transform(image1_pil)
        image2_resized = transform(image2_pil)

        # Convert images to tensors
        image1_tensor = transforms.ToTensor()(image1_resized)
        image2_tensor = transforms.ToTensor()(image2_resized)

        # Calculate LPIPS distance
        lpips_distance = loss_fn(image1_tensor, image2_tensor).item()

        # Accumulate the LPIPS values
        total_lpips += lpips_distance

    # Calculate the average LPIPS value
    average_lpips = total_lpips / num_pairs

    print("Average LPIPS value:", average_lpips)
else:
    print("No image pairs found.")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\dgner\anaconda3\lib\site-packages\lpips\weights\v0.1\alex.pth
Average LPIPS value: 0.02506647292863239


In [64]:
import os
import cv2
import glob
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Path to the directory containing the original dataset
original_dataset_dir = './data2'

# Path to the directory where the resized dataset will be saved
resized_dataset_dir = './data3'

# Define the desired size for the resized images
desired_size = (512, 512)

# Create the resized dataset directory if it doesn't exist
os.makedirs(resized_dataset_dir, exist_ok=True)

# Get the list of files in the original dataset directory
file_list = os.listdir(original_dataset_dir)

# Resize each image in the dataset
for file_name in file_list:
    # Construct the paths for the original and resized images
    original_image_path = os.path.join(original_dataset_dir, file_name)
    resized_image_path = os.path.join(resized_dataset_dir, file_name)

    # Load the original image
    image = cv2.imread(original_image_path)

    # Resize the image
    resized_image = cv2.resize(image, desired_size)

    # Save the resized image
    cv2.imwrite(resized_image_path, resized_image)

print("Dataset resized successfully.")



# Path to the directories containing the images of the two datasets
dataset1_dir = './data1'
dataset2_dir = './data3'

# Get the paths of the images in the two datasets
dataset1_images = glob.glob(dataset1_dir + '/*.png')
dataset2_images = glob.glob(dataset2_dir + '/*.png')


# Calculate SSIM for each pair of images
total_ssim = 0.0
num_pairs = min(len(dataset1_images), len(dataset2_images))

if num_pairs > 0:
    for i in range(num_pairs):
        # Load images
        image1 = cv2.imread(dataset1_images[i])
        image2 = cv2.imread(dataset2_images[i])

        # Check if images have the same shape
        if image1.shape != image2.shape:
            print(f"Skipping image pair {dataset1_images[i]} and {dataset2_images[i]} due to different dimensions.")
            continue

        # Resize images to a common size
        common_size = (256, 256)
        image1_resized = cv2.resize(image1, common_size)
        image2_resized = cv2.resize(image2, common_size)

        # Convert images to grayscale
        image1_gray = cv2.cvtColor(image1_resized, cv2.COLOR_BGR2GRAY)
        image2_gray = cv2.cvtColor(image2_resized, cv2.COLOR_BGR2GRAY)

        # Calculate SSIM
        ssim_value = ssim(image1_gray, image2_gray, data_range=image2_gray.max() - image2_gray.min())

        # Accumulate the SSIM values
        total_ssim += ssim_value

    # Calculate the average SSIM value
    average_ssim = total_ssim / num_pairs

    print("Average SSIM value:", average_ssim)
else:
    print("No image pairs found.")


Dataset resized successfully.
Average SSIM value: 0.9564099817922215


In [65]:
import os
import cv2
import glob
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio


# Path to the directory containing the original dataset
original_dataset_dir = './data2'

# Path to the directory where the resized dataset will be saved
resized_dataset_dir = './data3'

# Define the desired size for the resized images
desired_size = (512, 512)

# Create the resized dataset directory if it doesn't exist
os.makedirs(resized_dataset_dir, exist_ok=True)

# Get the list of files in the original dataset directory
file_list = os.listdir(original_dataset_dir)

# Resize each image in the dataset
for file_name in file_list:
    # Construct the paths for the original and resized images
    original_image_path = os.path.join(original_dataset_dir, file_name)
    resized_image_path = os.path.join(resized_dataset_dir, file_name)

    # Load the original image
    image = cv2.imread(original_image_path)

    # Resize the image
    resized_image = cv2.resize(image, desired_size)

    # Save the resized image
    cv2.imwrite(resized_image_path, resized_image)

print("Dataset resized successfully.")



# Path to the directory containing the original images
original_dir = './data1'

# Path to the directory containing the reconstructed/resized images
reconstructed_dir = './data3'

# Get the list of files in the original images directory
file_list = os.listdir(original_dir)

# Calculate PSNR for each pair of images
total_psnr = 0.0
num_images = 0

for file_name in file_list:
    # Construct the paths for the original and reconstructed images
    original_image_path = os.path.join(original_dir, file_name)
    reconstructed_image_path = os.path.join(reconstructed_dir, file_name)

    # Load the original and reconstructed images
    image1 = cv2.imread(original_image_path)
    image2 = cv2.imread(reconstructed_image_path)
    
     # Check if images have the same shape
    if image1.shape != image2.shape:
        print(f"Skipping image pair {dataset1_images[i]} and {dataset2_images[i]} due to different dimensions.")
        continue

    # Resize images to a common size
    common_size = (256, 256)
    image1_resized = cv2.resize(image1, common_size)
    image2_resized = cv2.resize(image2, common_size)
        

    # Convert the images to grayscale
    original_gray = cv2.cvtColor(image1_resized, cv2.COLOR_BGR2GRAY)
    reconstructed_gray = cv2.cvtColor(image2_resized, cv2.COLOR_BGR2GRAY)

    # Calculate PSNR
    psnr_value = peak_signal_noise_ratio(original_gray, reconstructed_gray, data_range=255)
    total_psnr += psnr_value
    num_images += 1

# Calculate average PSNR
average_psnr = total_psnr / num_images

print("Average PSNR:", average_psnr)

Dataset resized successfully.
Average PSNR: 35.08245133365495


In [66]:
import os
import glob
import numpy as np
from PIL import Image
from pytorch_fid import fid_score

# Path to the directory containing real images
real_images_dir = './data1'

# Path to the directory containing generated images
generated_images_dir = './data3'

# Get the paths of the images in the two datasets
dataset1_images = glob.glob(os.path.join(real_images_dir, '*.png'))
dataset2_images = glob.glob(os.path.join(generated_images_dir, '*.png'))

# Calculate FID
try:
    fid_value = fid_score.calculate_fid_given_paths(
        dataset1_images, dataset2_images, device='cpu', dims=2048
    )
    print("FID value:", fid_value)
except ValueError as e:
    print("FID calculation error:", e)
except Exception as e:
    print("An error occurred during FID calculation:", e)

0it [00:02, ?it/s]
0it [00:02, ?it/s]

FID calculation error: array must not contain infs or NaNs
